

# Movie Industry Data Analysis Project Group 1

This notebook analyzes movie data to answer key business questions for a new movie studio.
## Business Problem

Our company is planning to enter the film industry and launch a new movie studio. However, with no prior experience in movie production, we need to conduct thorough data analysis to guide our decisions. 

We aim to answer the following business questions:

1️⃣ What genres tend to perform best at the box office?  
2️⃣ Do critic ratings predict box office success?  
3️⃣ How do production budgets correlate with box office revenue?  
  

By answering these, we aim to make data-driven decisions on genre selection, budget allocation, and release scheduling.


## Table of Contents

- [1. Data Loading](#Data-Loading)
- [2. Data Cleaning](#Data-Cleaning)
- [3. Merging Datasets](#Merging-Datasets)
- [4. Exploratory Data Analysis](#Exploratory-Data-Analysis)
- [5. Business Recommendations](#Business-Recommendations)


## 1. Data Loading

We load all required datasets into pandas dataframes from both SQLite database and CSV/TSV files. These contain IMDB data, budget data, box office revenues, and Rotten Tomatoes info.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import pandas as pd
import re

# Load IMDB database tables
conn = sqlite3.connect("im.db")
df_basics = pd.read_sql_query("SELECT * FROM movie_basics", conn)
df_ratings = pd.read_sql_query("SELECT * FROM movie_ratings", conn)

# Load external CSV & TSV files
df_bom = pd.read_csv("bom.movie_gross.csv")
df_budgets = pd.read_csv("tn.movie_budgets.csv")
df_rt_info = pd.read_csv("rt.movie_info.tsv", sep="\t", encoding="latin1")


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

## 2. Data Cleaning

Each dataset has different formats, missing values, and currency strings. We'll clean them one by one to prepare for merging.

### Clean IMDB movie_basics

We clean movie_basics by:

- Converting year and runtime to numeric (handle non-numeric values).
- Replacing missing genres with 'Unknown'.
- Creating a lowercase title field for future merging across datasets.


In [3]:

df_basics['start_year'] = pd.to_numeric(df_basics['start_year'], errors='coerce')
df_basics['runtime_minutes'] = pd.to_numeric(df_basics['runtime_minutes'], errors='coerce')
df_basics['genres'] = df_basics['genres'].replace('\\N', pd.NA).fillna('Unknown')
df_basics['clean_title'] = df_basics['primary_title'].str.lower().str.strip()


### Clean IMDB movie_ratings

We convert ratings & vote counts to numeric and filter out movies with very few votes (under 50) to ensure we're using stable ratings.


In [4]:

df_ratings['averagerating'] = pd.to_numeric(df_ratings['averagerating'], errors='coerce')
df_ratings['numvotes'] = pd.to_numeric(df_ratings['numvotes'], errors='coerce')
df_ratings = df_ratings[df_ratings['numvotes'] >= 50]


### Clean BOM movie_gross

We convert the revenue columns to numeric and clean titles for merging.


In [5]:

df_bom['domestic_gross'] = pd.to_numeric(df_bom['domestic_gross'], errors='coerce')
df_bom['foreign_gross'] = pd.to_numeric(df_bom['foreign_gross'], errors='coerce')
df_bom['clean_title'] = df_bom['title'].str.lower().str.strip()


### Clean TN movie_budgets

We remove currency symbols and commas from budget fields, convert dates to datetime format, and extract release years.


In [6]:

for col in ['production_budget', 'domestic_gross', 'worldwide_gross']:
    df_budgets[col] = (
        df_budgets[col]
        .replace(r'[\$,]', '', regex=True)
        .replace('', np.nan)  # Replace empty strings with NaN
    )
    df_budgets[col] = pd.to_numeric(df_budgets[col], errors='coerce')  # Safe conversion


### Clean Rotten Tomatoes movie_info

We parse dates, convert box office strings to numeric, and create a clean_title for joining (here we use ID for simplicity).


In [7]:

df_rt_info['theater_date'] = pd.to_datetime(df_rt_info['theater_date'], errors='coerce')
df_rt_info['box_office'] = df_rt_info['box_office'].replace('[\$,]', '', regex=True).astype(float)
df_rt_info['clean_title'] = df_rt_info['id'].astype(str)
